In [3]:
import os
import pandas as pd 

from app.podcasts.application import VespaApp

from vespa.io import VespaQueryResponse
with open("app/token.txt", "r") as f:
    token =  f.read()
os.environ['VESPA_CLOUD_SECRET_TOKEN'] = token

app = VespaApp(deploy=False, token=token)

In [4]:
df = pd.read_csv("app/podcasts/data/transcribed_podcasts.csv")
resumos = pd.read_csv("full_data.csv", sep=";")

#join df and resumos on index
df = df.join(resumos, how="inner")
df = df.reset_index()
df.head(10)

,index,link,episode,title,date_on_air,description,transcript,podcast_id,response
0,0,https://www.thisamericanlife.org/sites/default...,1,New Beginnings,"1 November 17, 1995",Our program's very first broadcast.,Joe Franklin?\nI'm ready.\nIt's Ira Glass here...,1,"Kevin Kelly, a young man from New Jersey, wa..."
1,1,https://www.thisamericanlife.org/sites/default...,2,Small Scale Sin,"2 November 24, 1995",Small-scale stories on the nature of small-sca...,"OK, three boys, aged 13, 15, and 16. All three...",2,Eli was a hacker who belonged to a group cal...
2,2,https://www.thisamericanlife.org/sites/default...,3,Poultry Slam 1995,"3 December 1, 1995","Stories decrying the wonders of turkeys, chick...","In Danielle's house, ever since she was a girl...",3,Danielle's family has a tradition of servi...
3,3,https://www.thisamericanlife.org/sites/default...,4,Vacations,"4 December 8, 1995",Stories about vacations gone awry—or perhaps v...,The thing about Hawaii is that before you go a...,4,"Hitchhiking can be a dangerous, and the na..."
4,4,https://www.thisamericanlife.org/sites/default...,5,Anger and Forgiveness,"5 December 15, 1995","Stories that reveal the societal ""trend"" towar...",Hi. It's Ira Glass recording this in 2006. An...,5,The host and guests discuss the concept of f...
5,5,https://www.thisamericanlife.org/sites/default...,7,Quitting,"7 December 29, 1995",Stories of people who quit everything in their...,"What will make you change? For most of us, it ...",6,Evan Harris is a woman who decides to quit h...
6,6,https://www.thisamericanlife.org/sites/default...,8,New Year,"8 January 3, 1996",Stories that reflect back on 1995.,What's the French electrical supply like over...,7,The story is about a young girl named Claudi...
7,7,https://www.thisamericanlife.org/sites/default...,9,Julia Sweeney,"9 January 10, 1996",When comedian Julia Sweeney and her brother bo...,"OK, here's something that most of us would ne...",8,"Julia Sweeney, a comedian, was diagnosed wit..."
8,8,https://www.thisamericanlife.org/sites/default...,10,Double Lives,"10 January 17, 1996","People living their lives, just like the rest ...",There's what we wish for and there's what we g...,9,"Susan Bergman wrote a book about her father,..."
9,9,https://www.thisamericanlife.org/sites/default...,11,Enemies,"11 January 24, 1996",A story of a friendship between two adolescent...,"""I'll pour this pestilence into his ear. So w...",10,"In the first act, the narrator describes how..."


In [5]:
for i in range(0, len(df)):
    if 'Sharon' in df['response'][i]:
        print(i)

# sharon substitution
df.loc[157, "response"]= '    1. There\'s a subset of families in the US where children imitate their mothers all the time, even into adulthood.  2. Jonnafer\'s mother, Shirley Glass, is a psychologist in Baltimore. She recalls the time when her father left when she was a child, and how she had to take care of her three children.   3. Jonnafer\'s mother would talk in a country accent, and Jonna would imitate her by saying ""Sharon, Sharon, Sharon, please , Sharon, Sharon, Sharon, I know you hear me talking to you, Sharon, Sharon, get up, Sharon, I\'m going" '

157
498
512


In [6]:
from vespa.io import VespaQueryResponse
soma_m1 = 0
soma_m2 = 0
soma_m3 = 0
count = 0

###################################################################
#################           TESTE MV             ##################
###################################################################


with app.app.syncio(connections=1) as session:
    
    ################# SEMANTIC-FULL TITLE ######################
    for i in range(0, len(df)):
        query = df["title"][i]
        response = session.query(
                yql="select * from sources * where ({targetHits:1000}nearestNeighbor(embedding_full,q)) ",
                query=query,
                ranking="semantic_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
            )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id'])["id"].astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_semantic_full_title: {soma_m1/count}\n")
        f.write(f"hr1_semantic_full_title: {soma_m2/count}\n")
        f.write(f"mrr_semantic_full_title: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0
    ########################################################

    ################# SEMANTIC-FULL-MV TITLE #######################
    for i in range(0, len(df)):
        query = df["title"][i]
        
        response = session.query(
                yql="select * from sources * where ({targetHits:1000}nearestNeighbor(embedding_MV,q)) ",
                query=query,
                ranking="semantic_MV_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
            )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_semantic_MV_full_title: {soma_m1/count}\n")
        f.write(f"hr1_semantic_MV_full_title: {soma_m2/count}\n")
        f.write(f"mrr_semantic_MV_full_title: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0
    ########################################################

    ################# SEMANTIC-FULL RESUMO LLM ######################
    for i in range(0, len(df)):
        query = df["response"][i]
        response = session.query(
                yql="select * from sources * where ({targetHits:1000}nearestNeighbor(embedding_full,q)) ",
                query=query,
                ranking="semantic_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
            )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_semantic_full_resumo: {soma_m1/count}\n")
        f.write(f"hr1_semantic_full_resumo: {soma_m2/count}\n")
        f.write(f"mrr_semantic_full_resumo: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0
    ########################################################


    ################# SEMANTIC-FULL-MV RESUMO LLM #######################
    for i in range(0, len(df)):
        query = df["response"][i]
        
        response = session.query(
                yql="select * from sources * where ({targetHits:1000}nearestNeighbor(embedding_MV,q)) ",
                query=query,
                ranking="semantic_MV_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
            )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id'])["id"].astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_semantic_MV_full_resumo: {soma_m1/count}\n")
        f.write(f"hr1_semantic_MV_full_resumo: {soma_m2/count}\n")
        f.write(f"mrr_semantic_MV_full_resumo: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0
    ########################################################


    ################# FUSION-FULL TITLE ######################
    for i in range(0, len(df)):
        query = df["title"][i]
        response = session.query(
                yql="select * from sources * where userQuery() or ({targetHits:1000}nearestNeighbor(embedding_full,q)) limit 10",
                query=query,
                ranking="fusion_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
        )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id'])["id"].astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_fusion_full_title: {soma_m1/count}\n")
        f.write(f"hr1_fusion_full_title: {soma_m2/count}\n")
        f.write(f"mrr_fusion_full_title: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0  
    ########################################################


    ################# FUSION-FULL-MV TITLE ######################
    for i in range(0, len(df)):
        query = df["title"][i]
        response = session.query(
                yql="select * from sources * where userQuery() or ({targetHits:1000}nearestNeighbor(embedding_MV,q)) limit 10",
                query=query,
                ranking="fusion_MV_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
        )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_fusion_MV_full_title: {soma_m1/count}\n")
        f.write(f"hr1_fusion_MV_full_title: {soma_m2/count}\n")
        f.write(f"mrr_fusion_MV_full_title: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0
    ########################################################
    

    # ################# FUSION-FULL RESUMO LLM ######################
    for i in range(0, len(df)):
        query = df["response"][i]
        response = session.query(
                yql="select * from sources * where userQuery() or ({targetHits:1000}nearestNeighbor(embedding_full,q)) limit 10",
                query=query,
                ranking="fusion_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
        )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_fusion_full_resumo: {soma_m1/count}\n")
        f.write(f"hr1_fusion_full_resumo: {soma_m2/count}\n")
        f.write(f"mrr_fusion_full_resumo: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0
    ########################################################

    ################# FUSION-FULL-MV RESUMO LMM ######################
    for i in range(0, len(df)):
        query = df["response"][i]
        response = session.query(
                yql="select * from sources * where userQuery() or ({targetHits:1000}nearestNeighbor(embedding_MV,q)) limit 10",
                query=query,
                ranking="fusion_MV_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
        )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_fusion_MV_full_resumo: {soma_m1/count}\n")
        f.write(f"hr1_fusion_MV_full_resumo: {soma_m2/count}\n")
        f.write(f"mrr_fusion_MV_full_resumo: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0
    ########################################################
    

0
100
0
100
0
100
0
100
0
100
0
100
0
100
0
100


In [8]:
with app.app.syncio(connections=1) as session:

    ################# BM25 FULL RESUMO LLM ##########################
    # for i in range(0, len(df)):
    #     query = df["response"][i]
    #     response = session.query(
    #         yql="select * from sources * where userQuery() ",
    #         query=query,
    #         ranking="bm25_full",
    #         timeout="3000ms",
    #         hits=10,
    #     )
    #     if i == 0 or i == 100:
    #         print(i)
    #     assert response.is_successful()
    #     #for m1 we add 1 if in top10
    #     #for m2 we add 1 if in top1
    #     #for m3 we add the position in the ranking (if not in top10 we add 11)

    #     res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
    #     count += 1
    #     if df["episode"][i] in res:
    #         soma_m1 += 1
    #         if res.index(df["episode"][i]) == 0:
    #             soma_m2 += 1
    #         soma_m3 += (1/(res.index(df["episode"][i]) + 1))
    #     else:
    #         soma_m3 += 1/11
    # #write results to file
    # with open("results.txt", "a") as f:
    #     f.write(f"hr10_bm25_full_resumo: {soma_m1/count}\n")
    #     f.write(f"hr1_bm25_full_resumo: {soma_m2/count}\n")
    #     f.write(f"mrr_bm25_full_resumo: {soma_m3/count}\n")
    #     f.write(f"count: {count}\n")
    #     f.write("\n")
    # soma_m1 = 0
    # soma_m2 = 0
    # soma_m3 = 0
    # count = 0

    ################# BM25 FULL TITLE #######################
    for i in range(0, len(df)):
        query = df["title"][i]
        
        response =  session.query(
                yql="select * from sources * where userQuery() ",
                query=query,
                ranking="bm25_full",
                timeout="3000ms",
            )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()
        res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_bm25_full_title: {soma_m1/count}\n")
        f.write(f"hr1_bm25_full_title: {soma_m2/count}\n")
        f.write(f"mrr_bm25_full_title: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
        f.write("\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0

    ################# BM25 FULL DESCRIPTION ##########################
    for i in range(0, len(df)):
        query = df["description"][i]
        response = session.query(
            yql="select * from sources * where userQuery() ",
            query=query,
            ranking="bm25_full",
            timeout="3000ms",
            hits=10,
        )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()
        #for m1 we add 1 if in top10
        #for m2 we add 1 if in top1
        #for m3 we add the position in the ranking (if not in top10 we add 11)

        res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_bm25_full_description: {soma_m1/count}\n")
        f.write(f"hr1_bm25_full_description: {soma_m2/count}\n")
        f.write(f"mrr_bm25_full_description: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
        f.write("\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0

    # ################# FUSION-FULL DESCRIPTION ######################
    for i in range(0, len(df)):
        query = df["description"][i]
        response = session.query(
                yql="select * from sources * where userQuery() or ({targetHits:1000}nearestNeighbor(embedding_full,q)) limit 10",
                query=query,
                ranking="fusion_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
        )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_fusion_full_description: {soma_m1/count}\n")
        f.write(f"hr1_fusion_full_description: {soma_m2/count}\n")
        f.write(f"mrr_fusion_full_description: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
        f.write("\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0
    ########################################################

    ################# FUSION-FULL-MV DESCRIPTION ######################
    for i in range(0, len(df)):
        query = df["description"][i]
        response = session.query(
                yql="select * from sources * where userQuery() or ({targetHits:1000}nearestNeighbor(embedding_MV,q)) limit 10",
                query=query,
                ranking="fusion_MV_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
        )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_fusion_MV_full_description: {soma_m1/count}\n")
        f.write(f"hr1_fusion_MV_full_description: {soma_m2/count}\n")
        f.write(f"mrr_fusion_MV_full_description: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
        f.write("\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0

    ################# SEMANTIC-FULL DESCRIPTION ######################
    for i in range(0, len(df)):
        query = df["description"][i]
        response = session.query(
                yql="select * from sources * where ({targetHits:1000}nearestNeighbor(embedding_full,q)) ",
                query=query,
                ranking="semantic_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
            )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_semantic_full_description: {soma_m1/count}\n")
        f.write(f"hr1_semantic_full_description: {soma_m2/count}\n")
        f.write(f"mrr_semantic_full_description: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
        f.write("\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0
    ########################################################


    ################# SEMANTIC-FULL-MV DESCRIPTION #######################
    for i in range(0, len(df)):
        query = df["description"][i]
        
        response = session.query(
                yql="select * from sources * where ({targetHits:1000}nearestNeighbor(embedding_MV,q)) ",
                query=query,
                ranking="semantic_MV_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
            )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id'])["id"].astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_semantic_MV_full_description: {soma_m1/count}\n")
        f.write(f"hr1_semantic_MV_full_description: {soma_m2/count}\n")
        f.write(f"mrr_semantic_MV_full_description: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
        f.write("\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0

0
100


AttributeError: 'DataFrame' object has no attribute 'id'

In [10]:
from vespa.io import VespaQueryResponse
soma_m1 = 0
soma_m2 = 0
soma_m3 = 0
count = 0
with app.app.syncio(connections=1) as session:

    ################# BM25 FULL RESUMO LLM ##########################
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0
    with app.app.syncio(connections=1) as session:
        for i in range(0, len(df)):
            query = df["response"][i]
            response = session.query(
                yql="select * from sources * where userQuery() ",
                query=query,
                ranking="bm25_full",
                timeout="3000ms",
                hits=10,
            )
            if i == 0 or i == 100:
                print(i)
            assert response.is_successful()
            #for m1 we add 1 if in top10
            #for m2 we add 1 if in top1
            #for m3 we add the position in the ranking (if not in top10 we add 11)

            res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
            count += 1
            if df["episode"][i] in res:
                soma_m1 += 1
                if res.index(df["episode"][i]) == 0:
                    soma_m2 += 1
                soma_m3 += (1/(res.index(df["episode"][i]) + 1))
            else:
                soma_m3 += 1/11
        #write results to file
        with open("results.txt", "a") as f:
            f.write(f"hr10_bm25_full_resumo: {soma_m1/count}\n")
            f.write(f"hr1_bm25_full_resumo: {soma_m2/count}\n")
            f.write(f"mrr_bm25_full_resumo: {soma_m3/count}\n")
            f.write(f"count: {count}\n")

    ################# SEMANTIC-FULL RESUMO LLM ######################
    # for i in range(0, len(df)):
    #     query = df["response"][i]
    #     response = session.query(
    #             yql="select * from sources * where ({targetHits:1000}nearestNeighbor(embedding_full,q)) ",
    #             query=query,
    #             ranking="semantic_full",
    #             body={"input.query(q)": f"embed({query})"},
    #             timeout="3000ms",
    #         )
    #     if i == 0 or i == 100:
    #         print(i)
    #     assert response.is_successful()

    #     res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
    #     count += 1
    #     if df["episode"][i] in res:
    #         soma_m1 += 1
    #         if res.index(df["episode"][i]) == 0:
    #             soma_m2 += 1
    #         soma_m3 += (1/(res.index(df["episode"][i]) + 1))
    #     else:
    #         soma_m3 += 1/11
    # #write results to file
    # with open("results.txt", "a") as f:
    #     f.write(f"hr10_semantic_full_resumo: {soma_m1/count}\n")
    #     f.write(f"hr1_semantic_full_resumo: {soma_m2/count}\n")
    #     f.write(f"mrr_semantic_full_resumo: {soma_m3/count}\n")
    #     f.write(f"count: {count}\n")
    # soma_m1 = 0
    # soma_m2 = 0
    # soma_m3 = 0
    # count = 0
    # ########################################################

    # ################# FUSION-FULL RESUMO LLM ######################
    # for i in range(0, len(df)):
    #     query = df["response"][i]
    #     response = session.query(
    #             yql="select * from sources * where userQuery() or ({targetHits:1000}nearestNeighbor(embedding_full,q)) limit 10",
    #             query=query,
    #             ranking="fusion_full",
    #             body={"input.query(q)": f"embed({query})"},
    #             timeout="3000ms",
    #     )
    #     if i == 0 or i == 100:
    #         print(i)
    #     assert response.is_successful()

    #     res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
    #     count += 1
    #     if df["episode"][i] in res:
    #         soma_m1 += 1
    #         if res.index(df["episode"][i]) == 0:
    #             soma_m2 += 1
    #         soma_m3 += (1/(res.index(df["episode"][i]) + 1))
    #     else:
    #         soma_m3 += 1/11
    # #write results to file
    # with open("results.txt", "a") as f:
    #     f.write(f"hr10_fusion_full_resumo: {soma_m1/count}\n")
    #     f.write(f"hr1_fusion_full_resumo: {soma_m2/count}\n")
    #     f.write(f"mrr_fusion_full_resumo: {soma_m3/count}\n")
    #     f.write(f"count: {count}\n")
    # soma_m1 = 0
    # soma_m2 = 0
    # soma_m3 = 0
    # count = 0
    ########################################################

    
    # START TITLE

    ################# BM25 TITLE#######################
    # for i in range(0, len(df)):
    #     query = df["title"][i]
        
    #     response =  session.query(
    #             yql="select * from sources * where userQuery() ",
    #             query=query,
    #             ranking="bm25_full",
    #             timeout="3000ms",
    #         )
    #     if i == 0 or i == 100:
    #         print(i)
    #     assert response.is_successful()
    #     res = app.hits_as_df(response, ['id'])["id"].astype(int).tolist()
    #     count += 1
    #     if df["episode"][i] in res:
    #         soma_m1 += 1
    #         if res.index(df["episode"][i]) == 0:
    #             soma_m2 += 1
    #         soma_m3 += (1/(res.index(df["episode"][i]) + 1))
    #     else:
    #         soma_m3 += 1/11
    # #write results to file
    # with open("results.txt", "a") as f:
    #     f.write(f"hr10_bm25_full_title: {soma_m1/count}\n")
    #     f.write(f"hr1_bm25_full_title: {soma_m2/count}\n")
    #     f.write(f"mrr_bm25_full_title: {soma_m3/count}\n")
    #     f.write(f"count: {count}\n")
    # soma_m1 = 0
    # soma_m2 = 0
    # soma_m3 = 0
    # count = 0

    ################# FUSION-FULL TITLE ######################
    for i in range(0, len(df)):
        query = df["title"][i]
        response = session.query(
                yql="select * from sources * where userQuery() or ({targetHits:1000}nearestNeighbor(embedding_full,q)) limit 10",
                query=query,
                ranking="fusion_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
        )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id'])["id"].astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_fusion_full_title: {soma_m1/count}\n")
        f.write(f"hr1_fusion_full_title: {soma_m2/count}\n")
        f.write(f"mrr_fusion_full_title: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0  
    ########################################################

    ################# SEMANTIC-FULL TITLE######################
    for i in range(0, len(df)):
        query = df["title"][i]
        response = session.query(
                yql="select * from sources * where ({targetHits:1000}nearestNeighbor(embedding_full,q)) ",
                query=query,
                ranking="semantic_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
            )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id'])["id"].astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_semantic_full_title: {soma_m1/count}\n")
        f.write(f"hr1_semantic_full_title: {soma_m2/count}\n")
        f.write(f"mrr_semantic_full_title: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0
    ########################################################



    # END TITLE


    
    ################# SEMANTIC-FULL-MV  #######################
    for i in range(0, len(df)):
        query = df["response"][i]
        
        response = session.query(
                yql="select * from sources * where ({targetHits:1000}nearestNeighbor(embedding_MV,q)) ",
                query=query,
                ranking="semantic_MV_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
            )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id'])["id"].astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_semantic_MV_full_resumo: {soma_m1/count}\n")
        f.write(f"hr1_semantic_MV_full_resumo: {soma_m2/count}\n")
        f.write(f"mrr_semantic_MV_full_resumo: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0
    ########################################################

    ################# FUSION-FULL-MV RESUMO LMM ######################
    for i in range(0, len(df)):
        query = df["response"][i]
        response = session.query(
                yql="select * from sources * where userQuery() or ({targetHits:1000}nearestNeighbor(embedding_MV,q)) limit 10",
                query=query,
                ranking="fusion_MV_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
        )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_fusion_MV_full_resumo: {soma_m1/count}\n")
        f.write(f"hr1_fusion_MV_full_resumo: {soma_m2/count}\n")
        f.write(f"mrr_fusion_MV_full_resumo: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0
    ########################################################
    
    ################# FUSION-FULL-MV TITLE ######################
    for i in range(0, len(df)):
        query = df["title"][i]
        response = session.query(
                yql="select * from sources * where userQuery() or ({targetHits:1000}nearestNeighbor(embedding_MV,q)) limit 10",
                query=query,
                ranking="fusion_MV_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
        )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_fusion_MV_full_title: {soma_m1/count}\n")
        f.write(f"hr1_fusion_MV_full_title: {soma_m2/count}\n")
        f.write(f"mrr_fusion_MV_full_title: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0
    ########################################################
    


    ################# SEMANTIC-FULL-MV TITLE #######################
    for i in range(0, len(df)):
        query = df["title"][i]
        
        response = session.query(
                yql="select * from sources * where ({targetHits:1000}nearestNeighbor(embedding_MV,q)) ",
                query=query,
                ranking="semantic_MV_full",
                body={"input.query(q)": f"embed({query})"},
                timeout="3000ms",
            )
        if i == 0 or i == 100:
            print(i)
        assert response.is_successful()

        res = app.hits_as_df(response, ['id']).id.astype(int).tolist()
        count += 1
        if df["episode"][i] in res:
            soma_m1 += 1
            if res.index(df["episode"][i]) == 0:
                soma_m2 += 1
            soma_m3 += (1/(res.index(df["episode"][i]) + 1))
        else:
            soma_m3 += 1/11
    #write results to file
    with open("results.txt", "a") as f:
        f.write(f"hr10_semantic_MV_full_title: {soma_m1/count}\n")
        f.write(f"hr1_semantic_MV_full_title: {soma_m2/count}\n")
        f.write(f"mrr_semantic_MV_full_title: {soma_m3/count}\n")
        f.write(f"count: {count}\n")
    soma_m1 = 0
    soma_m2 = 0
    soma_m3 = 0
    count = 0
    ########################################################

0
100
0
100


HTTPError: 500 Server Error: Internal Server Error for url: https://bad202d2.b20230ac.z.vespa-app.cloud//search/?yql=select+%2A+from+sources+%2A+where+userQuery%28%29+or+%28%7BtargetHits%3A1000%7DnearestNeighbor%28embedding_full%2Cq%29%29+limit+10&query=See+No+Evil&ranking=fusion_full&timeout=3000ms

In [124]:
from vespa.io import VespaQueryResponse
soma_m1 = 0
soma_m2 = 0
soma_m3 = 0
count = 0
with app.app.syncio(connections=1) as session:

0
100


KeyboardInterrupt: 

In [68]:
print("m1: ", soma_m1/count)
print("m2: ", soma_m2/count)
print("m3: ", soma_m3/count)

m1:  1.0
m2:  0.8
m3:  1.4


In [61]:
#for every episode in df iterate over rs and pair "episode" with list of "id"
df["metric_calc"] = None
for i in range(0, len(df)):
    print(i)
    df.loc[i, "metric_calc"] = rs[i]['id'].tolist()


0


ValueError: Must have equal len keys and value when setting with an iterable

In [7]:
rs = []
for i in range(0, len(df)):
    response = app.query(df["response"][i], type_query = "bm25", fields = "full" , MV = False ,  return_df = True)
    rs.append(response)
    print(i)

0


KeyboardInterrupt: 